Translating lizon's posts to ENglish in order to apply Google Pegasus Summarization. Translation is more or less Ok but summarization is stil extractive, not abstractive. Need more efforts.

In [11]:
import os
import datetime
import pandas as pd
import numpy as np

In [23]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

In [16]:
mname = "facebook/wmt19-ru-en"

In [4]:
lizon_ids=[69715,
300514,
100899,
424711,
472696,
475085,
711697,
731728]

In [12]:
Data='/home/kate/Projects/eva/Data'

Messages_filename='Main/Messages.csv'
Messages_full_filename=os.path.join(Data, Messages_filename)

OriginalMessages_filename='Main/OriginalMessages.csv'
OriginalMessages_full_filename=os.path.join(Data, OriginalMessages_filename)

Authors_filename='Main/Authors.csv'
Authors_full_filename=os.path.join(Data, Authors_filename)

ta_extension_filename='Main/TextAttributes_ext.csv'
ta_extension_full_filename=os.path.join(Data, ta_extension_filename)


ft_filename='lizon_data_for_finetuning.csv'
ft_full_filename=os.path.join(Data, ft_filename)

ft_filename_test='Subprojects/lizon/lizon_clone_data_for_test.csv'
ft_full_filename_test=os.path.join(Data, ft_filename_test)

SimilarAuthors_filename='Subprojects/lizon/lizon_similar_authors.csv'
SimilarAuthors_full_filename=os.path.join(Data, SimilarAuthors_filename)

all_lizon_messages_filename='Subprojects/lizon/all_lizon_and_clones_messages.csv'
all_lizon_messages_full_filename=os.path.join(Data, all_lizon_messages_filename)

lizon_topics_filename='Subprojects/lizon/lizon_topics.csv'
lizon_topics_full_filename=os.path.join(Data, lizon_topics_filename)

In [13]:
lizon_topics = pd.read_csv(lizon_topics_full_filename, error_bad_lines=False, index_col=False)

/home/kate/anaconda3/envs/transformer/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
lizon_topics.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,author,Topic_Id,Topic,Chapter_Id,Chapter,...,message,cnt_childs,cnt_immediate_childs,message_words,Topic0,Score0,Topic1,Score1,Topic2,Score2
0,56799126,2010-05-12 22:03:00,N,0,69715,lizon **,2281736,как часто вы покупаете ребенку игрушки?,137,Детская психология и развитие,...,"Каждую неделю, на барахолках, где-то раз в мес...",1,1,21,NaN,NaN,NaN,NaN,NaN,NaN
1,57921402,2010-07-01 12:38:00,N,0,69715,lizon **,2322960,про негра,137,Детская психология и развитие,...,"Хе, а у нас, где мы жили раньше(свят свят, что...",0,0,71,фотка,0.728377,сын,0.620459,колледж,0.408833
2,58468210,2010-08-02 14:42:00,N,0,69715,lizon **,2350112,Как вы ходите с детьми в рестораны?,137,Детская психология и развитие,...,Начали ходить с дитем в рестораны лет с четыр...,0,0,16,NaN,NaN,NaN,NaN,NaN,NaN
3,58468241,2010-08-02 14:43:00,N,0,69715,lizon **,2351751,как спят ваши дети в жару,137,Детская психология и развитие,...,Укладываю в жару спать в открытой палатке в са...,0,0,14,NaN,NaN,NaN,NaN,NaN,NaN
4,58795095,2010-08-19 22:47:00,Y,0,69715,lizon **,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,...,У меня муж очень верующий католик. Я атеистка....,50,17,110,муж,0.817741,беременность,0.686566,религия,0.574169


In [14]:
AMessages = pd.read_csv(all_lizon_messages_full_filename, error_bad_lines=False, index_col=False)

In [11]:
AMessages=AMessages[AMessages['Author_Id'].isin(lizon_ids)]

In [4]:
AMessages.head(10)

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,Topic_Id,Topic,Chapter_Id,Chapter,original_message,message,cnt_childs,cnt_immediate_childs,author,eng_message
0,56799126,2010-05-12 22:03:00,N,0,69715,2281736,как часто вы покупаете ребенку игрушки?,137,Детская психология и развитие,"<div class=""body"">Каждую неделю, на барахолках...","Каждую неделю, на барахолках, где-то раз в мес...",1,1,lizon **,"Every week, in the sheds, about once a month, ..."
1,57921402,2010-07-01 12:38:00,N,0,69715,2322960,про негра,137,Детская психология и развитие,"<div class=""body"">Хе, а у нас, где мы жили ран...","Хе, а у нас, где мы жили раньше(свят свят, что...",0,0,lizon **,"And our eldest son, in high school, in French,..."
2,58468210,2010-08-02 14:42:00,N,0,69715,2350112,Как вы ходите с детьми в рестораны?,137,Детская психология и развитие,"<div class=""body"">Начали ходить с дитем в рест...",Начали ходить с дитем в рестораны лет с четыр...,0,0,lizon **,They started going to restaurants with their c...
3,58468241,2010-08-02 14:43:00,N,0,69715,2351751,как спят ваши дети в жару,137,Детская психология и развитие,"<div class=""body"">Укладываю в жару спать в отк...",Укладываю в жару спать в открытой палатке в са...,0,0,lizon **,I lay in the heat sleeping in an open tent in ...
4,58795443,2010-08-19 22:58:00,N,0,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">У меня страх что вдруг если ...",У меня страх что вдруг если такое вот рассказы...,11,9,lizon **,My fear is that if such a story were to be tol...
5,58806863,2010-08-20 13:44:00,N,0,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">Я думаю, в принципе знать и...","Я думаю, в принципе знать историю Христиансва...",11,2,lizon **,"I think, in principle, it is necessary to know..."
6,58807563,2010-08-20 14:08:00,N,0,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">Не совсем так.<br/> Я бы пре...",Не совсем так. Я бы предпочла чтоб мой сын(про...,0,0,lizon **,Not quite so. I would prefer my son (about who...
7,58808279,2010-08-20 14:32:00,N,0,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">Пока нет, но собирается зап...","Пока нет, но собирается записывать на катешиз...",1,1,lizon **,"They are such groups, usually on Wednesday mor..."
8,58808405,2010-08-20 14:36:00,N,58807966,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">Ну я имею в виду, Библию нуж...","Ну я имею в виду, Библию нужно знать для общей...",4,1,lizon **,"Well, I mean, you need to know the Bible for g..."
9,58808470,2010-08-20 14:37:00,N,58807819,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">А как гибко. К примеру, муж ...","А как гибко. К примеру, муж уже порассказал чт...",3,3,lizon **,"For example, my husband has already told me th..."


In [17]:
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

Downloading:   0%|          | 0.00/776k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/639k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/396k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/67.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

In [25]:
def ToEnglish(row):
    input = row['message']
    input_ids = tokenizer.encode(input, return_tensors="pt")
    outputs = model.generate(input_ids)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [27]:
AMessages['eng_message'] = AMessages.apply(ToEnglish, axis=1)

Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 1024). Running this sequence through the model will result in indexing errors


In [28]:
AMessages[['message','eng_message']]

,message,eng_message
7992,"Каждую неделю, на барахолках, где-то раз в мес...","Every week, in the sheds, about once a month, ..."
7993,"Хе, а у нас, где мы жили раньше(свят свят, что...","And our eldest son, in high school, in French,..."
7994,Начали ходить с дитем в рестораны лет с четыр...,They started going to restaurants with their c...
7995,Укладываю в жару спать в открытой палатке в са...,I lay in the heat sleeping in an open tent in ...
7996,У меня страх что вдруг если такое вот рассказы...,My fear is that if such a story were to be tol...
...,...,...
135811,"Ну да, я неправильно задала вопрос. Удивительн...","Well, yes, I was wrong to ask the question. It..."
135812,"Слушайте, если дело было у вас на даче, вам на...","Listen, if you had a case in your dacha, you h..."
135813,В Южной Америке крокодилы не водятся. Впрочем ...,Crocodiles are not found in South America. But...
135814,А вы живете в России или в другой стране? Во ...,Do you live in Russia or in another country? I...


In [17]:
AMessages.to_csv(all_lizon_messages_full_filename, header=True, index=False)

In [11]:
AMessages.columns

Index(['Message_Id', 'Timestamp', 'Topic_1st_Message', 'Parent_Id',
       'Author_Id', 'Topic_Id', 'Topic', 'Chapter_Id', 'Chapter',
       'original_message', 'message', 'cnt_childs', 'cnt_immediate_childs',
       'author', 'eng_message', 'Topic0', 'Score0', 'Topic1', 'Score1',
       'Topic2', 'Score2'],
      dtype='object')

In [15]:
AMessages = pd.merge(AMessages,lizon_topics[['Message_Id', 'Topic0', 'Score0', 'Topic1', 'Score1', 'Topic2','Score2']], left_on='Message_Id', right_on='Message_Id', how='inner' )

In [16]:
AMessages.head()

,Message_Id,Timestamp,Topic_1st_Message,Parent_Id,Author_Id,Topic_Id,Topic,Chapter_Id,Chapter,original_message,...,cnt_childs,cnt_immediate_childs,author,eng_message,Topic0,Score0,Topic1,Score1,Topic2,Score2
0,56799126,2010-05-12 22:03:00,N,0,69715,2281736,как часто вы покупаете ребенку игрушки?,137,Детская психология и развитие,"<div class=""body"">Каждую неделю, на барахолках...",...,1,1,lizon **,"Every week, in the sheds, about once a month, ...",NaN,NaN,NaN,NaN,NaN,NaN
1,57921402,2010-07-01 12:38:00,N,0,69715,2322960,про негра,137,Детская психология и развитие,"<div class=""body"">Хе, а у нас, где мы жили ран...",...,0,0,lizon **,"And our eldest son, in high school, in French,...",фотка,0.728377,сын,0.620459,колледж,0.408833
2,58468210,2010-08-02 14:42:00,N,0,69715,2350112,Как вы ходите с детьми в рестораны?,137,Детская психология и развитие,"<div class=""body"">Начали ходить с дитем в рест...",...,0,0,lizon **,They started going to restaurants with their c...,NaN,NaN,NaN,NaN,NaN,NaN
3,58468241,2010-08-02 14:43:00,N,0,69715,2351751,как спят ваши дети в жару,137,Детская психология и развитие,"<div class=""body"">Укладываю в жару спать в отк...",...,0,0,lizon **,I lay in the heat sleeping in an open tent in ...,NaN,NaN,NaN,NaN,NaN,NaN
4,58795443,2010-08-19 22:58:00,N,0,69715,2368698,читаь молитву на ночь в 5 лет-не слишком ли?,137,Детская психология и развитие,"<div class=""body"">У меня страх что вдруг если ...",...,11,9,lizon **,My fear is that if such a story were to be tol...,фотка,0.473151,пиано,0.156613,развод,0.113898


In [16]:
len(AMessages[((AMessages['Topic0']=='сын') | (AMessages['Topic1']=='сын') | (AMessages['Topic2']=='сын'))])

582

In [20]:
for index, row in AMessages[((AMessages['Topic0']=='сын') | (AMessages['Topic1']=='сын') | (AMessages['Topic2']=='сын'))].iterrows(): 
    print(row['eng_message'])

And our eldest son, in high school, in French, was once asked to write "I'm proud because I'm black." But this is the same essay, that I'm proud of what a white person would think of as racism. Is it fair? I then wrote an indignant letter in the lyceum.
I think, in principle, it is necessary to know the history of Christianity, at least for the general culture (all classical painting, according to the Bible). I will not say that my husband is a fanatic, but according to observations, his blind faith does not really help him, on the contrary.
Not quite so. I would prefer my son (about whom I speak, the youngest) to be less idealistic in life than my husband, da.Less idealistic is easier to live with. Of course, there are qualities in a husband that I do not like, I think it is rare to meet a wife who is satisfied with absolutely everything in her husband, this has no influence on love.
Well, I mean, you need to know the Bible for general culture, at least schematically. My youngest son 

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/pegasus-reddit_tifu")

model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-reddit_tifu")



Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/87.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [10]:
text='My first son, Andrey, is 18 years old. He is a colledge student. My second son is a high school student. Ron is 16 years old. The name of my third son is Alex. he is in a kindergarden. My daughter was just born. She does not have a name yet'
inputs = tokenizer(text, return_tensors='pt')
prediction = model.generate(**inputs)
prediction = tokenizer.batch_decode(prediction)
print(prediction)

["<pad> my son's name is Andrey. my son's name is Ron. my son's name is Alex. my daughter's name is not yet known.</s>"]
